# Individual Trackers

In [ ]:
import pandas as pd;
import numpy as np;
import os;
import time;
import datetime;
import warnings;
warnings.filterwarnings('ignore');
from pandas.api.types import CategoricalDtype;

## SPI_POL_DISH_Tracker

### Line Level

In [ ]:
#Reading the required Line level excel file
Dish_data=pd.read_excel('//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/03. Raw Data/Connect Data/SPI POL DISH Tracker/POL_Dish_Line.xlsx')


In [ ]:
#Renaming the Column names 
Dish_data.rename(columns={"Periods":"Facts"},inplace=True)

**Removing unwanted characters from all the column names** and store them in a list ***new_col_names***

In [ ]:
new_col_names=[]
for i in Dish_data.columns:
    x=len(i)
    if x<=17:
        new_col_names.append(i)
    elif i[:3]=='YTD':
        new_col_names.append(i[:-16].strip('- '))
    else:
        new_col_names.append(i[:-14].strip('- '))

new_col_names[7:9]=["FY 2020","FY 2019"]

Renaming the column names

In [ ]:
# Line_Monthly.rename(columns=dict(zip(Line_Monthly.columns,new_col_names)),inplace=True)
Dish_data.columns=new_col_names
Dish_data.columns

**Splitting "Products" column**

Creating 6 new columns **"Mega Category","ShareBasis","Manufacturer","Franchise","Brand","SubBrand"** from **"Products"** column by splitting 

In [ ]:
Dish_data[['Mega Category','Category','ShareBasis','Manufacturer','Franchise','Size']] = Dish_data['Products'].str.split('|',n=5,expand=True)


***movecol*** function to **rearranging** the columns of a dataframe

In [ ]:
def movecol(df, cols_to_move=[], ref_col='', place=''):
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [ ]:
# Moving columns after "Market"
Dish_data = movecol(Dish_data, 
             cols_to_move=['ShareBasis','Manufacturer','Franchise','Size'], 
             ref_col='Markets',
             place='After')

#Dropping the columns not required in the output files
Dish_data.drop(["Category","Mega Category"],axis=1,inplace=True)


**Replacing all the None values** in columns **"ShareBasis","Manufacturer","Franchise","Size"**

In [ ]:
Dish_data[['ShareBasis','Manufacturer','Franchise','Size']]=Dish_data[['ShareBasis','Manufacturer','Franchise','Size']].fillna(value="")


In [ ]:
# adding a "KEY" column
Dish_data["Key"]=Dish_data["Markets"]+Dish_data["Products"]+Dish_data["Facts"]

In [ ]:
# moving "KEY" column to start of the column
Dish_data = movecol(Dish_data, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')
Dish_data.head(3)

In [ ]:
#sorting dataframe by markets, products, facts order
Dish_data=Dish_data.sort_values(['Markets', 'Products','Facts'],ascending=[True,True,True])

#removing duplicates rows
Dish_data.drop_duplicates(subset="Key",keep='last',inplace=True)

#replacing nan values from zero
Dish_data.fillna(0,inplace=True)

In [ ]:
#Exporting the file in CSV format
Dish_data.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/SPI_POL_DISH_Tracker Data.csv",index=False)

### UPC LEVEL 

In [ ]:
#Reading the required UPC level excel file
Dish_data_UPC=pd.read_excel('//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/03. Raw Data/Connect Data/SPI POL DISH Tracker/POL_Dish_UPC.xlsx')


**Splitting "Products" column**

Creating 6 new columns **"Mega Category", "Concentrate","Category","ShareBasis", "Manufacturer","Franchise", "Size","Brand","SubBrand","SubBrand Variant","Item","UPC"** from **"Products"** column by splitting 

In [ ]:
Dish_data_UPC[['Mega Category','Concentrate','Category','ShareBasis','Manufacturer','Franchise','Size','Brand','SubBrand','SubBrand Variant','Item','UPC']] = Dish_data_UPC['Products'].str.split('|',n=11,expand=True)


In [ ]:
#Moving columns after the markets column
Dish_data_UPC = movecol(Dish_data_UPC, 
             cols_to_move=['Size','Item','UPC',"Brand","SubBrand","SubBrand Variant"], 
             ref_col='Markets',
             place='After')
#Dropping columns not required in the output file
Dish_data_UPC.drop(["Category","Mega Category","Concentrate"],axis=1,inplace=True)


In [ ]:
#Dropping columns not required in the output file
del Dish_data_UPC["ShareBasis"]
del Dish_data_UPC["Manufacturer"]
del Dish_data_UPC["Franchise"]

### Renaming the time period columns 

In [ ]:
connect_name=list(set(Dish_data_UPC["Periods"]))

new_name=[]
new_name2=[]
for i in connect_name:
    x=len(i)
    if x<=17:
        new_name.append(i)
    elif i[:3]=='YTD':
        new_name.append(i[:-16].strip('- '))
    else:
        new_name.append(i[:-14].strip('- '))
        
for i in new_name:
    
    if i=='Cal Yr 2019':
        new_name2.append('FY 2019')
    elif i=='Cal Yr 2020':
        new_name2.append('FY 2020')
    else:
        new_name2.append(i)

        
period_mapping = dict(zip(connect_name, new_name2)) 
Dish_data_UPC["Periods"].replace(period_mapping,inplace= True)

### Reordering the columns in as required in the output file

In [ ]:
fact_ordering=["$ Shr - CP MEGA CATEGORY","$ Shr Chg YA - CP MEGA CATEGORY",
                 "Base $ Shr - $ - CP MEGA CATEGORY","Base $ Shr Chg YA - $ - CP MEGA CATEGORY",
                 "Incr $ Shr - $ - CP MEGA CATEGORY","Incr $ Shr Chg YA - $ - CP MEGA CATEGORY",
                 "$","$ % Chg YA","Units","Units % Chg YA","EQ","EQ % Chg YA","%ACV","%ACV Chg YA","$ / $MM ACV","$ / $MM ACV % Chg YA",
                 "Units / $MM ACV","Units / $MM ACV % Chg YA","Avg EQ Price","Avg EQ Price % Chg YA",'Avg Unit Price','Avg Unit Price % Chg YA',"Base EQ Price",
                 "Base EQ Price % Chg YA","Any Promo EQ Price","Any Promo EQ Price % Chg YA","% EQ Any Promo","% EQ Any Promo Chg YA",
                 "Feat or Disp %ACV","Feat or Disp %ACV Chg YA","% $ Price Decr Only","% $ Price Decr Only Chg YA"
]

In [ ]:
column_ordering=['Markets', 'Size', 'Item', 'UPC', 'Brand', 'SubBrand',
       'SubBrand Variant', 'Periods']+fact_ordering

In [ ]:
Dish_data_UPC_final=Dish_data_UPC[column_ordering]

In [ ]:
#Adding dummy columns 
Dish_data_UPC_final["Container"]=""
Dish_data_UPC_final["Blank1"]=""
Dish_data_UPC_final["Blank2"]=""

In [ ]:
Dish_data_UPC_final = movecol(Dish_data_UPC_final, 
             cols_to_move=["Container","Blank1","Blank2"], 
             ref_col='SubBrand Variant',
             place='After')


In [ ]:
Dish_data_UPC_final["Level_2"]=Dish_data_UPC_final["Size"]
#Rordering the new columns
Dish_data_UPC_final = movecol(Dish_data_UPC_final, 
             cols_to_move=["Level_2"], 
             ref_col='Feat or Disp %ACV Chg YA',
             place='After')


In [ ]:
# Removing OZ from Size column and strip the column
Dish_data_UPC_final["Size"]=Dish_data_UPC_final["Size"].str.replace(" OZ","")
Dish_data_UPC_final["Size"]=Dish_data_UPC_final["Size"].apply(lambda x : x.strip())


**UPC Filter**

#### Filtering UPC's for Dollar sales value > 10000 for US xAOC in Latest 52 weeks

In [ ]:
Dish_data_UPC_final_backup=Dish_data_UPC_final.copy()

In [ ]:
df1=Dish_data_UPC_final[(Dish_data_UPC["Periods"]=="Latest 52 Wks") & (Dish_data_UPC_final["Markets"]=="Total US xAOC")]
df2=df1[df1['$']>10000]
df2.head(2)

In [ ]:
UPC=list(set(df2["UPC"]))+[' 003511001417',' 005800014029']

In [ ]:
Dish_data_UPC_final=Dish_data_UPC_final[Dish_data_UPC_final['SubBrand']!= ' POL ESSENTIAL CLEAN ']

In [ ]:
Dish_data_UPC_final=Dish_data_UPC_final[Dish_data_UPC_final["UPC"].isin(UPC)]


**SIZE Filter: Keeping only sizes required in the front end**


In [ ]:
Size = ["8","10","18","20","32.5","46","56","68.5","90","102"]

In [ ]:
Dish_data_UPC_final=Dish_data_UPC_final[Dish_data_UPC_final['Size'].isin(Size)]

In [ ]:
#sorting dataframe in markets, products, facts order
Dish_data_UPC_final=Dish_data_UPC_final.sort_values(['Markets', 'UPC','Periods'],ascending=[True,True,True])

#removing duplicates rows
Dish_data_UPC_final.drop_duplicates(subset=['Markets', 'UPC','Periods'],keep='last',inplace=True)

#replacing nan values from zero
Dish_data_UPC_final.fillna(0,inplace=True)


In [ ]:
Dish_data_UPC_final.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/SPI_POL_DISH_UPC.csv",index=False)

## OW Pen Tracker

## 1. Monthly and Weekly Data prep

**Accessing Input Files**

In [ ]:
monthly = pd.read_excel(r"\\falmumapp34\Nielsen Connect Transition & Maintenance (20-SCP-3650)\03. Raw Data\Connect Data\OW Pen Tracker\OW Pen Tracker Monthly.xlsx")
weekly= pd.read_excel(r"\\falmumapp34\Nielsen Connect Transition & Maintenance (20-SCP-3650)\03. Raw Data\Connect Data\OW Pen Tracker\OW Pen Tracker Weekly.xlsx")

**Changing column "Periods" to "Facts"**

In [ ]:
monthly=monthly.rename(columns={'Periods':'Facts'})
weekly=weekly.rename(columns={'Periods':'Facts'})

**Time period mapping**

**Manual Step**

This mapping needs to be changed every year when new year data arrives

In [ ]:
time_period_map= pd.DataFrame({'Connect Time Period':['Markets', 'Products', 'Facts','Cal Yr 2020','Cal Yr 2019','Cal Yr 2018','Cal Yr 2017','YTD','YTD YA','Latest 13 Wks','Latest 26 Wks','Dec 21','Nov 21','Oct 21','Sep 21','Aug 21','Jul 21','Jun 21','May 21','Apr 21','Mar 21','Feb 21','Jan 21','Dec 20','Nov 20','Oct 20','Sep 20','Aug 20','Jul 20','Jun 20','May 20','Apr 20','Mar 20','Feb 20','Jan 20','1 w/e','Launch To Date'],
                               'Final Name':['Markets', 'Products', 'Facts','FY 2020','FY 2019','FY 2018','FY 2017','YTD 2021','YTD 2020','Latest 13 Weeks','Latest 26 Weeks','Dec 21','Nov 21','Oct 21','Sep 21','Aug 21','Jul 21','Jun 21','May 21','Apr 21','Mar 21','Feb 21','Jan 21','Dec 20','Nov 20','Oct 20','Sep 20','Aug 20','Jul 20','Jun 20','May 20','Apr 20','Mar 20','Feb 20','Jan 20','Latest Week','Launch To Date']
                              ,'Order':range(1,38)})

In [ ]:
# assigning variables
ow_col = list(monthly.columns)
time_connect=list(time_period_map["Connect Time Period"])
time_final=list(time_period_map["Final Name"])

# changing column names based on time_period_mapping
new_column = []
for i in range(0,len(ow_col)):
    for j in range(0,len(time_connect)):
        if time_connect[j] in ow_col[i]:
            new_column.append(time_final[j])
                    
# assigning new column names to Line_Monthly
monthly.columns = new_column

creating ***movecol*** function to **rearranging** the columns of a dataframe

In [ ]:
def movecol(df, cols_to_move=[], ref_col='', place=''):
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [ ]:
# adding a "KEY" column
monthly["Key"]=monthly["Markets"]+monthly["Products"]+monthly["Facts"]
weekly["Key"]=weekly["Markets"]+weekly["Products"]+weekly["Facts"]

# moving "KEY" column to start of the column
monthly = movecol(monthly, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')

# moving "KEY" column to start of the column
weekly = movecol(weekly, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')

In [ ]:
# removing duplicate rows
weekly.drop_duplicates(inplace=True)
# removing duplicated rows
monthly.drop_duplicates(inplace=True)

**Exporting data into output files**

In [ ]:
# exporting weekly data
weekly.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/OW Pen Tracker Weekly.csv",index=False)

# exporting monthly data
monthly.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/OW Pen Tracker Monthly.csv",index=False)

## 2. LaunchBenchmark Data prep

**Accessing Input Files**

In [ ]:
# reading data
sort= pd.read_excel(r"\\falmumapp34\Nielsen Connect Transition & Maintenance (20-SCP-3650)\03. Raw Data\Connect Data\OW Pen Tracker\OW Pen Tracker YTD Sort.xlsx")
# reading data
launch_bm = pd.read_excel(r"\\falmumapp34\Nielsen Connect Transition & Maintenance (20-SCP-3650)\03. Raw Data\Connect Data\OW Pen Tracker\OW Pen Tracker Launch Benchmark.xlsx")

### a. YTD Sort Report

**Stripping the columns**

In [ ]:
cols=list(sort.columns[0:2])
sort[cols]=sort[cols].apply(lambda x : x.str.strip())

**Market Mapping**

In [ ]:
market_mapping={
    "Total US xAOC":"TOTAL US XAOC",
    "Total US Drug":"TOTAL US DRUG",
    "Total US Food":"TOTAL US FOOD",
    "Target Total BM/FF TA":"TARGET TOTAL BM/FF CENSUS TA",
    "Walmart Total US TA":"TOTAL WALMART TOTAL US CTA",
    "Walgreens Corp Total TA":"WALGREENS CORP TOTAL CENSUS TA",
    "ShopRite Total TA":"SHOPRITE TOTAL CENSUS TRADING AREA",
    "Meijer Total TA":"MEIJER TOTAL CENSUS TRADING AREA",
    "CVS Total Corp WO HI TA":"CVS TOTAL CORP WO HI CENSUS TA",
    "Ahold USA Corp Total TA":"AHOLD USA CORP TOTAL CTA",
    "Wegmans Total TA":"WEGMAN'S TOTAL CENSUS TRADING AREA"
}

**Products Mapping**

In [ ]:
product_mapping={
    'WHITENING SYS | WS PEN | ACTIVE WOW | REM SUBBRAND | ACTIVE WOW TOOTH WHITENER CHARCOAL PLUS MINT WS PEN .09 FLUID OUNCE REM MULTI':"ACTIVE WOW WS PEN CHRC PLS MNT .09 FLD OZ 001",
    "WHITENING SYS | WS PEN | COLGATE-PALMOLIVE | COLGATE OPTIC WHITE PEN | COLGATE OPTIC WHT PEN OVERNGHT TOOTH WHITENER WS PEN .08 FLUID OUNCE REM MULTI":"COLGATE OPTIC WHITE PEN OVERNIGHT .08 FLD OZ 001",
    "WHITENING SYS | WS PEN | P&G | ARC PRECISION APPLICATOR | ARC TOOTH WHITENER WS PEN 0.13 FLUID OUNCE REM MULTI":"ARC PRECISION APPLICATOR WS PEN .13 FLD OZ 001",
    "WHITENING SYS | WS GEL | P&G | CREST WT EMLSN BUILT-IN APLCTR | CREST WHITENING TOOTH WHITENER WS GEL .35 OUNCE REM MULTI":"CREST WT EMLSN BUILT-IN APLCTR WGL .35 OZ 001",
    "WHITENING SYS | WS GEL | P&G | CREST WT EMLSN WAND APPLCTR + LIGHT | CREST WHITENING TOOTH WHITENING KIT WS GEL 1 COUNT REM MULTI":"CREST WT EMLSN WAND APPLCTR WGL 1 CT 001",
    "WHITENING SYS | WS PEN | REM MFG | MOON WHITENING PENS | MOON TOOTH WHITENER VANILLA MINT WS PEN .09 FLUID OUNCE REM MULTI":"MOON WS PEN VNL MNT .09 FLD OZ 001",
    "WHITENING SYS | WS PEN | REM MFG | REM SUBBRAND | BRITENZ TOOTH WHITENER MINTY FRESH TASTE WS PEN .5 FLUID OUNCE REM MULTI":"BRITENZ WS PEN MNTY FRSH TST .5 FLD OZ 001",
    "WHITENING SYS | WS PEN | REM MFG | REM SUBBRAND | BRITENZ TOOTH WHITENER MINTY FRESH TASTE WS PEN .05 FLUID OUNCE REM MULTI":"BRITENZ WS PEN MNTY FRSH TST .05 FLD OZ 003",
    "WHITENING SYS | WS PEN | REM MFG | REM SUBBRAND | DALE AUDREY WHITE 2 NITE TOOTH WHITENER PEPPER MINT WS PEN .07 OUNCE REM MULTI":"DALE AUDREY WHITE 2 NITE WS PEN PPR MNT .07 OZ 001",
    "WHITENING SYS | WS PEN | REM MFG | REM SUBBRAND | DR GINGER'S TOOTH WHITENER COCONUT MINT WS PEN NO FLUORIDE .35 OUNCE REM MULTI":"DR GINGER'S SUT WS PEN CCNT MNT .35 OZ 001",
    "WHITENING SYS | WS PEN | P&G | ARC TRAVEL APPLICATOR | ARC TOOTH WHITENER WS PEN .06 FLUID OUNCE REM MULTI":"ARC TRAVEL APPLICATOR WS PEN .06 FLD OZ 001",
    "WHITENING SYS | WS PEN | REM MFG | REM SUBBRAND | BRUSH BUDDIES TOOTH WHITENER WS PEN .08 FLUID OUNCE REM MULTI":"BRUSH BUDDIES WS PEN .08 FLD OZ 001",
    "WHITENING SYS | WS PEN | SMILE DIRECT CLUB | SML DRCT CLB WHTNG KIT":"SMILE DIRECT CLUB WHITENING KIT 8 PENS",
    "WHITENING SYS | WS PEN | PRVT LBL | PRVT LBL PEN | PRIVATE LABEL TOOTH WHITENER FRESH MINT GEL .11 FLUID OUNCE":"CTL BR PRVT LBL PEN WS PEN FRSH MNT .11 FLD OZ 001",
    "WHITENING SYS | WS PEN | PRVT LBL | PRVT LBL PEN | CVS HEALTH TOOTH WHITENER FRESH MINT WS PEN .08 FLUID OUNCE REM MULTI":"CTL BR PRVT LBL PEN WS PEN FRSH MNT .08 FLD OZ 001",
    "WHITENING SYS | WS PEN | P&G | GLEEM WHITENING PEN | GLEEM TOOTH WHITENER WS PEN 0.13 FLUID OUNCE REM MULTI":"GLEEM WHITENING PEN WS PEN .13 FLD OZ 001"
}

In [ ]:
# Applying Market and product mappping
sort["Markets"].replace(market_mapping,inplace=True)
sort["Products"].replace(product_mapping,inplace=True)


**Markets and Products Filter**

In [ ]:
# Market filter
sort_markets = ["TOTAL US XAOC","TOTAL US DRUG","TOTAL US FOOD","TARGET TOTAL BM/FF CENSUS TA","TOTAL WALMART TOTAL US CTA",
                "WALGREENS CORP TOTAL CENSUS TA","SHOPRITE TOTAL CENSUS TRADING AREA","MEIJER TOTAL CENSUS TRADING AREA",
                "CVS TOTAL CORP WO HI CENSUS TA","AHOLD USA CORP TOTAL CTA","WEGMAN'S TOTAL CENSUS TRADING AREA"]
# Products filter
sort_products = ["ACTIVE WOW WS PEN CHRC PLS MNT .09 FLD OZ 001","COLGATE OPTIC WHITE PEN OVERNIGHT .08 FLD OZ 001",
                 "ARC PRECISION APPLICATOR WS PEN .13 FLD OZ 001","CREST WT EMLSN BUILT-IN APLCTR WGL .35 OZ 001",
                 "CREST WT EMLSN WAND APPLCTR WGL 1 CT 001","MOON WS PEN VNL MNT .09 FLD OZ 001",
                 "BRITENZ WS PEN MNTY FRSH TST .5 FLD OZ 001","BRITENZ WS PEN MNTY FRSH TST .05 FLD OZ 003",
                 "DALE AUDREY WHITE 2 NITE WS PEN PPR MNT .07 OZ 001","DR GINGER'S SUT WS PEN CCNT MNT .35 OZ 001",
                 "ARC TRAVEL APPLICATOR WS PEN .06 FLD OZ 001","BRUSH BUDDIES WS PEN .08 FLD OZ 001",
                 "SMILE DIRECT CLUB WHITENING KIT 8 PENS","CTL BR PRVT LBL PEN WS PEN FRSH MNT .11 FLD OZ 001",
                 "CTL BR PRVT LBL PEN WS PEN FRSH MNT .08 FLD OZ 001","GLEEM WHITENING PEN WS PEN .13 FLD OZ 001"]

In [ ]:
# Applying Filters
sort= sort[sort["Markets"].isin(sort_markets)]
sort=sort[sort["Products"].isin(sort_products)]

In [ ]:
# Renaming column "$" to column "Dollars"
sort.rename(columns = {"$":"Dollars"}, inplace = True)

# Removing duplicates
sort.drop_duplicates(inplace=True)

**Exporting data into output files**

In [ ]:
sort.to_csv("//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/OW Pen Tracker YTD Sort.csv",index=False)

## b. **Launch Benchmark Report**

**Changing column "Periods" to "Facts"**

In [ ]:
lbm=launch_bm.rename(columns={'Periods':'Facts'})

In [ ]:
# assigning variables
lbm_col = list(lbm.columns)
time_connect=list(time_period_map["Connect Time Period"])
time_final=list(time_period_map["Final Name"])

# changing column names based on time_period_mapping
new_column = []
for i in range(0,len(lbm_col)):
    for j in range(0,len(time_connect)):
        if time_connect[j] in lbm_col[i]:
            new_column.append(time_final[j])
            
# assigning new column names to Lbm dataframe
lbm.columns = new_column

**Stripping the columns**

In [ ]:
cols=list(lbm.columns[0:2])

lbm[cols]=lbm[cols].apply(lambda x : x.str.strip())

**Fact Mapping**

In [ ]:
fact_mapping={
    "$":"Dollars",
    "Base $":"Baseline $",
    "Incr $":"Incr.$",
    "Units":"Units",
    "%ACV":"% Stores Selling",
    "TDP":"Cumulative Distribution Pts",
    "Avg Unit Price":"RASP",
    "$ / %ACV":"$ per Pt Distribution",
    "$ Shr - CP CATEGORY":"$ Share AHW",
    "$ Shr - CP FORM":"$ Share AHW Pen"
}

In [ ]:
# applying "Markets","Products" and "Facts" mapping
lbm["Markets"].replace(market_mapping,inplace=True)
lbm["Products"].replace(product_mapping,inplace=True)
lbm["Facts"].replace(fact_mapping,inplace=True)

In [ ]:
# Creating "Key" column
lbm["Key"] = lbm["Markets"]+lbm["Products"]+lbm["Facts"]

# moving "KEY" column to start of the column
lbm = movecol(lbm, 
             cols_to_move=['Key'], 
             ref_col='Markets',
             place='Before')

**Markets, Facts, Products Selection**

In [ ]:
lbm_markets = ["TOTAL US XAOC","TOTAL US DRUG","TOTAL US FOOD","TARGET TOTAL BM/FF CENSUS TA","TOTAL WALMART TOTAL US CTA",
               "WALGREENS CORP TOTAL CENSUS TA","SHOPRITE TOTAL CENSUS TRADING AREA","MEIJER TOTAL CENSUS TRADING AREA",
               "CVS TOTAL CORP WO HI CENSUS TA","AHOLD USA CORP TOTAL CTA","WEGMAN'S TOTAL CENSUS TRADING AREA"]

lbm_facts = ["Dollars","Baseline $","Incr.$","Units","% Stores Selling","Cumulative Distribution Pts","RASP",
             "$ per Pt Distribution","$ Share AHW","$ Share AHW Pen"]

lbm_products = ["ACTIVE WOW WS PEN CHRC PLS MNT .09 FLD OZ 001","COLGATE OPTIC WHITE PEN OVERNIGHT .08 FLD OZ 001",
                "ARC PRECISION APPLICATOR WS PEN .13 FLD OZ 001","CREST WT EMLSN BUILT-IN APLCTR WGL .35 OZ 001",
                "CREST WT EMLSN WAND APPLCTR WGL 1 CT 001","MOON WS PEN VNL MNT .09 FLD OZ 001",
                "BRITENZ WS PEN MNTY FRSH TST .5 FLD OZ 001","BRITENZ WS PEN MNTY FRSH TST .05 FLD OZ 003",
                "DALE AUDREY WHITE 2 NITE WS PEN PPR MNT .07 OZ 001","DR GINGER'S SUT WS PEN CCNT MNT .35 OZ 001",
                "ARC TRAVEL APPLICATOR WS PEN .06 FLD OZ 001","BRUSH BUDDIES WS PEN .08 FLD OZ 001",
                "SMILE DIRECT CLUB WHITENING KIT 8 PENS","CTL BR PRVT LBL PEN WS PEN FRSH MNT .11 FLD OZ 001",
                "CTL BR PRVT LBL PEN WS PEN FRSH MNT .08 FLD OZ 001","GLEEM WHITENING PEN WS PEN .13 FLD OZ 001"]

In [ ]:
# appying filters 
lbm= lbm[lbm["Facts"].isin(lbm_facts)]
lbm= lbm[lbm["Markets"].isin(lbm_markets)]
lbm=lbm[lbm["Products"].isin(lbm_products)]

**Reading Previous month Launch Benchmark file**

In [ ]:
ow_hist_data = pd.read_csv(r"\\falmumapp34\Nielsen Connect Transition & Maintenance (20-SCP-3650)\05. Output Data\OW Pen Tracker Launch Benchmark.csv")

**Merge Launch Benchmark Data with Historical Data**

In [ ]:
# merge ow_hist_data and lbm
new = pd.merge(ow_hist_data,lbm, left_on = "KEY",right_on = "Key" ,how='left')

# Dropping Unwanted columns
new.drop(['Key', 'Markets', 'Products', 'Facts'], axis=1, inplace = True)

**Time Mapping** ( It is available till Dec 2026)

In [ ]:
last_column_mapping={
    "Nov 19":"11/23/2019",
    "Dec 19":"12/28/2019",
    "Jan 20":"02/01/2020",
    "Feb 20":"02/29/2020",
    "Mar 20":"04/04/2020",
    "Apr 20":"05/02/2020",
    "May 20":"05/30/2020",
    "Jun 20":"07/04/2020",
    "Jul 20":"08/01/2020",
    "Aug 20":"08/29/2020",
    "Sep 20":"10/03/2020",
    "Oct 20":"10/31/2020",
    "Nov 20":"11/28/2020",
    "Dec 20":"01/02/2021",
    "Jan 21":"01/30/2021",
    "Feb 21":"02/27/2021",
    "Mar 21":"04/03/2021",
    "Apr 21":"05/01/2021",
    "May 21":"05/29/2021",
    "Jun 21":"07/03/2021",
    "Jul 21":"07/31/2021",
    "Aug 21":"08/28/2021",
    "Sep 21":"10/02/2021",
    "Oct 21":"10/30/2021",
    "Nov 21":"11/27/2021",
    "Dec 21":"01/01/2022",
    "Jan 22":"01/29/2022",
    "Feb 22":"02/26/2022",
    "Mar 22":"04/02/2022",
    "Apr 22":"04/30/2022",
    "May 22":"05/28/2022",
    "Jun 22":"06/25/2022",
    "Jul 22":"07/23/2022",
    "Aug 22":"08/20/2022",
    "Sep 22":"09/17/2022",
    "Oct 22":"10/15/2022",
    "Nov 22":"11/12/2022",
    "Dec 22":"12/10/2022",
    "Jan 23":"01/07/2023",
    "Feb 23":"02/04/2023",
    "Mar 23":"03/04/2023",
    "Apr 23":"04/01/2023",
    "May 23":"04/29/2023",
    "Jun 23":"05/27/2023",
    "Jul 23":"06/24/2023",
    "Aug 23":"07/22/2023",
    "Sep 23":"08/19/2023",
    "Oct 23":"09/16/2023",
    "Nov 23":"10/14/2023",
    "Dec 23":"11/11/2023",
    "Jan 24":"12/09/2023",
    "Feb 24":"01/06/2024",
    "Mar 24":"02/03/2024",
    "Apr 24":"03/02/2024",
    "May 24":"03/30/2024",
    "Jun 24":"04/27/2024",
    "Jul 24":"05/25/2024",
    "Aug 24":"06/22/2024",
    "Sep 24":"07/20/2024",
    "Oct 24":"08/17/2024",
    "Nov 24":"09/14/2024",
    "Dec 24":"10/12/2024",
    "Jan 25":"11/09/2024",
    "Feb 25":"12/07/2024",
    "Mar 25":"01/04/2025",
    "Apr 25":"02/01/2025",
    "May 25":"03/01/2025",
    "Jun 25":"03/29/2025",
    "Jul 25":"04/26/2025",
    "Aug 25":"05/24/2025",
    "Sep 25":"06/21/2025",
    "Oct 25":"07/19/2025",
    "Nov 25":"08/16/2025",
    "Dec 25":"09/13/2025",
    "Jan 26":"10/11/2025",
    "Feb 26":"11/08/2025",
    "Mar 26":"12/06/2025",
    "Apr 26":"01/03/2026",
    "May 26":"01/31/2026",
    "Jun 26":"02/28/2026",
    "Jul 26":"03/28/2026",
    "Aug 26":"04/25/2026",
    "Sep 26":"05/23/2026",
    "Oct 26":"06/20/2026",
    "Nov 26":"07/18/2026",
    "Dec 26":"08/15/2026"
}

In [ ]:
# applying Time Mapping ( changing last column name) 
new.rename(columns = last_column_mapping, inplace = True)

# removing duplicate rows
new.drop_duplicates(inplace=True)

**Exporting Data into output Folder**

In [ ]:
new.to_csv("\\falmumapp34\Nielsen Connect Transition & Maintenance (20-SCP-3650)\05. Output Data\OW Pen Tracker Launch Benchmark.csv",index=False)

## Individual Tracker Ends